## Loading Datasets In

In [3]:
import pandas as pd

attacking_df = pd.read_csv('attacking.csv')
attempts_df = pd.read_csv('attempts.csv')
defending_df = pd.read_csv('defending.csv')
disciplinary_df = pd.read_csv('disciplinary.csv')
distribution_df = pd.read_csv('distribution.csv')
goals_df = pd.read_csv('goals.csv')
key_stats_df = pd.read_csv('key_stats.csv')